## Draw boxes on images from xml annotation files

In [54]:
import xml.etree.ElementTree as ET
import os
import numpy as np


def draw_box_xml(img, anno):
    """Draw box on image
    
    Args:
        img(ndarray): image file array
        anno(xml.etree.ElementTree.Element): annotation element
        
    Returns:
        img(ndarray): image with box array
    """
    for obj in anno.findall('object'):
        x1 = int(float(obj.find('robndbox').find('x1').text))
        x2 = int(float(obj.find('robndbox').find('x2').text))
        x3 = int(float(obj.find('robndbox').find('x3').text))
        x4 = int(float(obj.find('robndbox').find('x4').text))
        
        y1 = int(float(obj.find('robndbox').find('y1').text))
        y2 = int(float(obj.find('robndbox').find('y2').text))
        y3 = int(float(obj.find('robndbox').find('y3').text))
        y4 = int(float(obj.find('robndbox').find('y4').text))
                
        p1 = (x1, y1)
        p2 = (x2, y2)
        p3 = (x3, y3)
        p4 = (x4, y4)
        ps = np.int0(np.array([p1, p2, p3, p4]))
        
        box_color = (0, 0, 255) if obj.find('name').text == 'ship' else (255, 0, 0)
        cv2.drawContours(img, [ps], -1, box_color, thickness=1)
    
    return img
    

In [56]:
for anno_name in os.listdir('img_tif/annotations'):
    tree = ET.parse(os.path.join('img_tif', 'annotations', anno_name))
    root = tree.getroot()
    img_name = os.path.basename(root.find('path').text)
    img = cv2.imread(os.path.join('img_tif', 'patch', img_name))
    assert x is not None, 'Image not exists on the path: {}'.format(os.path.join('img_tif', 'patch', img_name))
    drawn_img = draw_box_xml(img, root)
    cv2.imwrite(os.path.join('img_tif', 'images_sample', img_name), drawn_img)

## Draw boxes on images from COCO annotation files

In [60]:
import json
import cv2
import os
import numpy as np
from pycocotools.coco import COCO

dataset = 'val'
annotation_path = os.path.join(dataset, f'{dataset}_annotations.json')
img_home = os.path.join(dataset, 'patch_img')
vis_home = os.path.join(dataset, 'patch_img_gt_vis')
box_color = (255, 0, 255)

os.makedirs(vis_home, exist_ok=True)

coco = COCO(annotation_path)

cnt = 0
ann_cnt = 0
for img_id in range(len(coco.getImgIds())):
    ann_ids =  coco.getAnnIds(imgIds=[img_id])
    if not ann_ids:
        continue
        
    img_name = coco.loadImgs(ids=[img_id])[0]['file_name']
    img_path = os.path.join(img_home, img_name)

    try:
        img = cv2.imread(img_path)
        cnt += 1
    except:
        print('File not exists: ', img_path)
        continue

    # Draw boxes
    for ann_id in ann_ids:
        bbox = coco.loadAnns(ids=[ann_id])[0]['bbox']
        p1 = bbox[0:2]
        p2 = bbox[2:4]
        p3 = bbox[4:6]
        p4 = bbox[6:8]
        ps = np.intp(np.array([p1, p2, p3, p4]))    
        drawn_img = cv2.drawContours(img, [ps], -1, box_color, thickness=1)
    cv2.imwrite(os.path.join(vis_home, img_name), drawn_img)

print('Total images: ', len(coco.getImgIds()))
print('Images saved: ', cnt)
print('Done')

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Total images:  539
Images saved:  100
Done
